In [1]:
import numpy as np

In [2]:
def period(npa):
    for a in range(1, len(npa) // 2):
        shift = np.roll(npa, a)
        if np.array_equal(npa[a:], shift[a:]):
            return a
    return None

In [3]:
def calc_cols(p, cmax, dmax, display):
    prev = np.array([1 for a in range(dmax)])
    cols = []
    for a in range(cmax):
        if display != 0:
            print(period(prev, dmax), "\t", prev[:display])
        cols.append(prev)
        prev = np.cumsum(prev) % p
    return cols

In [4]:
def calc_snps(cols, nmax, verbose=False):
    unique = [1]
    stacked = [None for a in range(len(cols))]
    # periods = [period(cols[a]) for a in range(len(cols))]
    periods = [None for a in range(len(cols))]
    reduced = [cols[a][:(nmax + periods[a])] if periods[a] != None else cols[a] for a in range(len(cols))]
    if verbose:
        print(1, end="")
    for n in range(1, nmax):
        snp = None
        for a in range(len(cols)):
            if stacked[a] is None:
                stacked[a] = np.array([reduced[a][nmax:]])
            else:
                stacked[a] = np.vstack((stacked[a], np.roll(reduced[a], n - 1)[nmax:]))
            if snp is None:
                snp = np.unique(stacked[a], axis=1)
            else:
                snp = np.unique(np.hstack((snp, stacked[a])), axis=1)
        unique.append(snp.shape[1])
        if verbose:
            print(",", snp.shape[1], end="")
    if verbose:
        print()
    return unique

In [ ]:
primes = [2, 3, 5, 7, 11, 13, 17, 19]
prime_snps = {}
for prime in primes:
    cdmax = prime**5
    cols = calc_cols(prime, cdmax, cdmax, 0)
    print("Prime:", prime)
    print("|S(n, p)|, 0 <= n <= p^3: ", end="")
    prime_snps[prime] = calc_snps(cols, prime**3 + 1, verbose=True)
    
    # fit quadratic with n=p^k for k=0,1,2
    xs = [1, prime, prime**2]
    ys = [prime_snps[prime][1], prime_snps[prime][prime], prime_snps[prime][prime**2]]
    quadratic = np.polynomial.polynomial.polyfit(xs, ys, 2)
    print("Quadratic:", quadratic)
    
    # test for n=p^3
    snp_p3 = np.polynomial.polynomial.polyval(prime**3, quadratic)
    print("n =", prime**3, ":\tExpected:", prime_snps[prime][prime**3], "\tActual:", snp_p3)
    print()

Prime: 2
|S(n, p)|, 0 <= n <= p^3: 1, 2, 4, 8, 14, 22, 32, 44, 58
Quadratic: [ 2. -1.  1.]
n = 8 :	Expected: 58 	Actual: 58.00000000000003

Prime: 3
|S(n, p)|, 0 <= n <= p^3: 1, 3, 9, 25, 43, 71, 109, 157, 207, 259, 313, 377, 451, 535, 629, 733, 847, 971